In [5]:

import dill
import panel as pn
import json
import altair as alt

with open('gdf_merged.dill', 'rb') as f:
    gdf_merged = dill.load(f)

variable_list = ['Apartment Construction/Maintenance', 'Building Maintenance',
                 'Heat/Hot Water/Electricity', 'Noise - Outside',
                 'Noise - Residential', 'Parking/Street Conditions', 'Rodent',
                 'Unhealthy Conditions', 'Water Leaks and Plumbing']
variable = pn.widgets.Select(options=variable_list, name='Variable')


@pn.depends(variable.param.value)
def get_map(variable):
    gdf_merged_complaint = gdf_merged[gdf_merged["Complaint Type"] == variable]
    choro_json = json.loads(gdf_merged_complaint.to_json())
    choro_data = alt.Data(values=choro_json['features'])
    return gen_map(geodata=choro_data, color_column='properties.count_col_yr',
                   tooltip=['properties.zipcode:O','properties.count_col_yr:Q'],
                   color_scheme='redblue').repeat(row=[variable])
    
def gen_map(geodata, color_column, tooltip, color_scheme):
    
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(stroke='white').encode(
        alt.Color(color_column,type='quantitative',
                  scale=alt.Scale(scheme=color_scheme, reverse = True),title = "counts per year"),
        tooltip=tooltip)
    return choro

pn.Row(
    pn.Column('# Complaint Type Map', pn.panel(variable)),
    get_map
).show()

Launching server at http://localhost:50499


In [2]:
with open('pipe_of_logreg.dill', 'rb') as f:
    lr_pipe = dill.load(f)

In [4]:
import pandas as pd
test={"Units":[25.0], "Stories":[6.0],"Built":[1907.0],"Popularity":[3.0], "zipcode":["10032"], "neighborhood":["Washington Heights"],"unit_type":["Rental Building"]}
test_df = pd.DataFrame.from_dict(test)
X_axis_probs = [round(x,2) for x in lr_pipe.predict_proba(test_df).flatten().tolist()]
Y_axis = lr_pipe.classes_.flatten().tolist()
predictions = pd.DataFrame(zip(Y_axis,X_axis_probs), columns = ["Complaint Type", "Probability"])
predictions.sort_values( by = "Probability", ascending = False)

,Complaint Type,Probability
2,Heat/Hot Water/Electricity,0.30
3,Noise - Outside,0.30
4,Noise - Residential,0.13
0,Apartment Construction/Maintenance,0.09
8,Water Leaks and Plumbing,0.08
1,Building Maintenance,0.04
5,Parking/Street Conditions,0.04
7,Unhealthy Conditions,0.02
6,Rodent,0.01
